# Gathering Data
Take your data and upload to Cloudant.

# Converting Data into CSV
The data in the database must be converted into a form that can be used to train the models for entity extraction and natural language classification. To do this, run the script below. Please edit the database connections so that they point to your database where all your data is stored.

In [ ]:
import cloudant
import csv

SERVER = 'https://1790ef54-fcf2-4029-9b73-9000dff88e6e-bluemix.cloudant.com' #Replace with your server URL
DATABASE = 'amazon_data' #Replace with the name of the database
USERNAME = '1790ef54-fcf2-4029-9b73-9000dff88e6e-bluemix' #Replace with the username from your credentials for the NLC
PASSWORD = '5beb3f8b9f95586542e3d9c5acfb0c52832252432623e534d4e88b12fad29638' #Replace with the password from your credentials for the NLC
DESIGN = '' #replace with the name of the design document that contains the view. This should be of the form '_design/XXXX'
VIEW = 'names/namesAndAsin' #Replace with the view from your database to poll, this should take the form of view_file/view and should return the text to classify as the value field and what you would like to call it as the key
DESTINATION = 'out.csv' #Replace with correct name for output file (NOTE must be *.csv)

server = cloudant.client.Cloudant(USERNAME,PASSWORD,url=SERVER)
server().connect
db = server[DATABASE]
query = db.get_view_result(DESIGN,VIEW)
file = open(DESTINATION, 'wb')
writer = csv.writer(file)


for q in query:
    print q[0]
    if 'key' in q[0] and q[0]['key'] != None:
        title = q[0]['key']
    else:
        title = "No Title"
    if 'value' in q[0] and q[0]['value'] != None:
        text = q[0]['value']
    else:
        text = "No Text"
    writer.writerow([title,text])

file.close()

# Import Data CSV into Watson Knowledge Studio
The CSV that is created at the end of the script must now be imported into Watson Knowledge Studio (WKS). Each row in the CSV will be treated as a separate document and will be organized in WKS. The documents then need to be annotated with entities and relationships. Coreference is also done here. A strict guideline is very helpful when doing this.

After the annotations are done and the model is trained, it needs to be exported into Alchemy Language using an API key. This API key is used below in the url variable. Please replace it in the variable.

In [ ]:
import requests
import ast
import pprint
pp = pprint.PrettyPrinter(depth=6)

def get_relations(review):
    url = "http://access.alchemyapi.com/calls/text/TextGetTypedRelations?showSourceText=1&model=8f03f142-af7f-4487-be47-371fc3262705&apikey=ffd7397f4be657f7740a84038f903271b2707a11&outputMode=json"
    f = requests.get(url, params={'text':review})
    response = f.content
    response = ast.literal_eval(response)
    return response

def get_entities(review)
    url = "http://access.alchemyapi.com/calls/text/TextGetRankedNamedEntities?showSourceText=1&model=8f03f142-af7f-4487-be47-371fc3262705&apikey=ffd7397f4be657f7740a84038f903271b2707a11&outputMode=json&sentiment=1"
    f = requests.get(url, params={'text':review})
    response = f.content
    response = ast.literal_eval(response)
    return response

def token_replacement_entities(reivew):
    review = get_entities(review)
    if 'entities' in review:
        entities = review['entities']
        text = review['text']
        for i in entities:
            token = i['text']
            count = i['count']
            classification = "<" + i['type'] + ">"
            text = text.replace(token,classification,count)
    return text

def token_replacement_relations(review):
    review = get_relations(review)
    if 'typedRelations' in review:
        types = review['typedRelations']
        text = review['text']
        standard = ""
        previous = ""
        new_sentence = True
        review_text_mod = ""
        flip = True
        sentence_dict={}
        prev_dict={}
        if types != []:
            count=0
            result=[]
            for sentence in types:
                #dict={}
                review_text = sentence['sentence']
                #dict['sentence']=review_text
                if review_text != standard:
                    previous = standard
                    standard = review_text
                    new_sentence = True
                    dict={}
                    sentence_dict[standard]=count
                    dict['sentence']=standard
                    dict['seqno']=count
                    print "\ndict\n"
                    print dict
                    text = text.replace(previous, review_text_mod, 1)
                    count+=1
                    #if count>1:
                    result.append(prev_dict)
                else:
                    new_sentence = False
                    dict=result[sentence_dict[review_text]]
                    review_text = review_text_mod
                temp_dict={}
                temp_dict['hasrel']=sentence['type']
                for entity in sentence['arguments']:
                    for i in entity['entities']:
                        token = i['text']
                        temp_dict[entity['part']]=token
                        if entity['part']=='first':
                            if i['type'] in dict:
                                dict[i['type']].append(temp_dict)
                            else:
                                dict[i['type']]=[]
                                dict[i['type']].append(temp_dict)
                        else:
                            sec_dict={}
                            sec_dict['name']=i['text']
                            if i['type'] in dict:
                                dict[i['type']].append(sec_dict)
                            else:
                                dict[i['type']]=[]
                                dict[i['type']].append(sec_dict)
                        classification = "<" + i['type'] + ">"
                        if flip:
                            review_text_mod = review_text.replace(token,classification,1)
                            flip = False
                        else:
                            review_text_mod = review_text_mod.replace(token,classification,1)
                            flip = True
                dict['reversed_sentence']=review_text_mod
                #print temp_dict
                print "final dict"
                print dict
                prev_dict=dict
                result[sentence_dict[sentence['sentence']]]=dict
                #result.append(dict)
                print "new"
                #print sentence_dict
        pp.pprint(result)
        text = text.replace(standard,review_text_mod,1)
        return text
        #pp.print(result
    else: 
        return review

# Replace entities with Tags in Training Sets
You will need to run your training set CSV through the script below which replaces any entities found by Entity Extraction in Alchemy language with a representative tag in order generalize the classifier in the coming steps. Just identify the name for your csv file input and output and it should work.

In [ ]:
import token_replacement as t
import csv

read = open('ground_truth_layer1.csv','rb') ##replace with correct filenames
write = open('ground_truth_layer1_replace.csv','wb')     ##replace with correct filenames

reader = csv.reader(read)
writer = csv.writer(write)

for row in reader:
    token = t.token_replacement(row[0])
    writer.writerow([token,row[1]])

read.close()
write.close()

# Train Natural Language Classifier
The Natural Language Classifier (NLC) needs to be trained on this generalized/tagged data. This is a 3 level NLC that will provide better classification results than just using a single layer. Add your credentials in the file below and run the script to train the NLC. The script will also store your classifier_id's in a json file.

In [ ]:
import couchdbkit
import json
from watson_developer_cloud import NaturalLanguageClassifierV1
import csv

tier1CSV = 'training_set1.csv'
tier2CSV = 'training_set2.csv'
tier3CSV = 'training_set3.csv'
USERNAME = 'e561bc30-d294-41f4-8b47-39fc6bc29917'
PASSWORD = 'XH8pYnsYfClv'
JSON_TARGET = '../../data/classifier_ids.json'

classifierTree = {
    'tier1':'',
    'tier2':'',
    'tier3':''
}

# Initialize classifier
nlc = NaturalLanguageClassifierV1(username = USERNAME, password = PASSWORD)

# Train tier 1 classifier
print("############# TIER 1 CLASSIFIER ##############")
with open(tier1CSV, 'rb') as training_data:
  classifier = nlc.create(
    training_data=training_data,
    name='tier1',
    language='en'
  )
print(json.dumps(classifier, indent=2))
classifierTree['tier1'] = classifier['classifier_id']

# Train tier 2 classifier
print("############# TIER 2 CLASSIFIER ##############")
with open(tier2CSV, 'rb') as training_data:
  classifier = nlc.create(
    training_data=training_data,
    name='tier2',
    language='en'
  )
print(json.dumps(classifier, indent=2))
classifierTree['tier2'] = classifier['classifier_id']

# Train tier 3 classifier
print("############# TIER 3 CLASSIFIER ##############")
with open(tier3CSV, 'rb') as training_data:
  classifier = nlc.create(
    training_data=training_data,
    name='tier3',
    language='en'
  )
print(json.dumps(classifier, indent=2))
classifierTree['tier3'] = classifier['classifier_id']

# Write the tiers with classifier id's to file for use later
with open(JSON_TARGET, 'w') as outfile:
    json.dump(classifierTree, outfile)

print("############# FULL CLASSIFIER TREE ##############")
print(json.dumps(classifierTree))